# <center> Exploring Pre-WWII Soviet Socialist Realism with Text Analysis

<img src="image.png">

About: This notebook uses logistic regression and topic modeling to explore the particularities of the Soviet literary genre of socialist realism in the pre-WII Soviet Union.  

The python modules nltk, gensim, pymystem, and pyLDAvis are used in this notebook.

In [4]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords 
import string
import gensim
import scipy
from gensim import corpora
from gensim import models
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt
from pymystem3 import Mystem
import pyLDAvis
import re

# Section 1 : Logistic Regression

This section uses predictive modeling using logistic regression. 

Results indicate that, logistic regression, with a certain degree of accuracy can differentiate between the socialist realism and banned literature corpora.  It is likely that accuracy might be even higher, were deep learning techniques to be applied. 

# Section 2: Topic Modeling

Import .csv with data from entire socialist realist literature corpus. 

In [2]:
litdata = pd.read_csv("csv_files/socialist_realism_for_tm.csv")
litdata['text'].head(5)

0     СОДЕРЖАНИЕ I. Морозка II . Мечик III . Шестое...
1     . Многие открыто роптали на `` подлость и нес...
2     бровей на тоненькую девушку с длинными косами...
3     время люди очнулись и поняли , что наступило ...
4     кричал Рябец , навалившись на одно слово и вс...
Name: text, dtype: object

Define stopwords, punctuation, read in output from named entity recognition (NER) script.

In [3]:
with open('NERoutput.txt') as fp:
    lines = fp.read().splitlines()

lines = [i.lower() for i in lines]

stop = list(stopwords.words('russian'))
stop.extend(['это', 'свой', 'то', ' ','который'])
stop.extend(lines)

punctuation = list(string.punctuation)
punctuation.extend(['--', '—', '«', '»'," -- ", "\n", "\r", "\t","-" ,"-", "...", "…", " - ", " « ", " ", "..", "``", "\"\"","\'\'"])


Use function to create a new column in pandas df with cleaned text.

In [4]:
# Create new column in dataframe that is literary texts without stopwords and punctuation, as well as without
# named entities for people, as identified in NERoutput.txt, which was created with NERscript.py

def cleantext(text):
    tokens = text.split(" ")
    tokenslower = [i.strip().lower() for i in tokens]
#the following list comprehension gets rid of word-final punctuation, while retaining word-medial punctuation in words like "кто-то"
    tokens_no_word_final_punc = [re.sub(r'[!|,|?|.|:|;]|\.{1,}$\)', "", item) for item in tokenslower]
    tokens_no_stop_or_punc = [i for i in tokens_no_word_final_punc if i not in stop if i not in punctuation]
#the following line gets rid of blank, empty spaces in each list    
    tokens_no_stop_or_punc = list(filter(None, tokens_no_stop_or_punc))
    return tokens_no_stop_or_punc
    
litdata["no_punc_or_stop"] = litdata["text"].apply(cleantext)
no_punc_or_stop = litdata["no_punc_or_stop"]

# Print first five rows of new column

no_punc_or_stop.head(5)

0    [содержание, i, ii, iii, шестое, чувство, iv, ...
1    [многие, открыто, роптали, подлость, несознани...
2    [бровей, тоненькую, девушку, длинными, косами,...
3    [время, люди, очнулись, поняли, наступило, утр...
4    [кричал, рябец, навалившись, одно, слово, веря...
Name: no_punc_or_stop, dtype: object

Use pymystem to lemmatize the text. 

In [2]:
mystem = Mystem() 


def lemmatize(text):
    joinedtokens = " ".join(text)
    lemmatized = mystem.lemmatize(joinedtokens)
    lemmatized = [i for i in lemmatized if i not in stop if i not in punctuation]
    return lemmatized
    
litdata["lemmatized"] = litdata["no_punc_or_stop"].apply(lemmatize)

litdata["lemmatized"].head(5)

NameError: name 'Mystem' is not defined

Topic modeling with gensim, using Latent Semantic Indexing (LSI)

In [6]:
dictionary = corpora.Dictionary(litdata["lemmatized"] )
doc_term_matrix = [dictionary.doc2bow(item) for item in litdata["lemmatized"] ]
lsi = models.LsiModel(doc_term_matrix, num_topics=4, id2word=dictionary)
output = lsi.print_topics(num_topics=4, num_words = 4)
print (output)

[(0, '0.286*"сказать" + 0.255*"человек" + 0.228*"рука" + 0.204*"говорить"'), (1, '0.324*"мать" + 0.322*"сказать" + -0.246*"казак" + 0.227*"человек"'), (2, '-0.203*"знать" + -0.183*"дело" + 0.177*"глаз" + 0.168*"голова"'), (3, '0.286*"казак" + -0.228*"человек" + 0.224*"мать" + 0.157*"говорить"')]


Results are skewed because of the length of certain works in the corpus, primarily "Quiet Flows the Don".  Next step, try to rectify these results by using a smaller, more even amount of text from each work in the corpus.

## What are the results with smaller chunked files?

In [7]:
import pandas as pd

newsocreal = pd.read_csv("csv_files/socreal_files_chunked_smaller.csv")
newsocreal.head(5)

,title,name,date,english metadata,source,text
0,Разгром,"Фадеев , Александр Александрович",1926,"Aleksandr Fadeev , The Rout , 1926",http : //lib.ru/RUSSLIT/FADEEW/razgrom.txt,СОДЕРЖАНИЕ I. Морозка II . Мечик III . Шестое...
1,\n\nРазгром\n\n,"Фадеев, Александр Александрович\n\n",2,"\n\nAleksandr Fadeev, The Rout, 1926\n\n",\n\nhttp://lib.ru/RUSSLIT/FADEEW/razgrom.txt\n\n,"пыльным , сумасшедшим карьером . -- Обожди-и ,..."
2,\n\nРазгром\n\n,"Фадеев, Александр Александрович\n\n",3,"\n\nAleksandr Fadeev, The Rout, 1926\n\n",\n\nhttp://lib.ru/RUSSLIT/FADEEW/razgrom.txt\n\n,.. -- снова повторил Дубов . -- Блуди ! Посмот...
3,\n\nРазгром\n\n,"Фадеев, Александр Александрович\n\n",4,"\n\nAleksandr Fadeev, The Rout, 1926\n\n",\n\nhttp://lib.ru/RUSSLIT/FADEEW/razgrom.txt\n\n,"Ведь надо как-то , умеют же другие ... '' В мы..."
4,\n\nРазгром\n\n,"Фадеев, Александр Александрович\n\n",5,"\n\nAleksandr Fadeev, The Rout, 1926\n\n",\n\nhttp://lib.ru/RUSSLIT/FADEEW/razgrom.txt\n\n,"ребят , вернувшихся из-за реки . Дубов узнал ,..."


Apply all of the same steps to the .csv with smaller chunks of each individual work in the corpus.  

In [8]:
with open('NERoutput.txt') as fp:
    lines = fp.read().splitlines()

lines = [i.lower() for i in lines]
lines = " ".join(lines)
mystem = Mystem() 
lemmatizedlines = mystem.lemmatize(lines)
lemmatizedlines = [i for i in lemmatizedlines if i != " "]

stop = list(stopwords.words('russian'))
stop.extend(['это', 'свой', 'то', ' ','который', 'говорить', 'сказать', 'знать', 'спрашивать', 'вопрос'])
stop.extend(lemmatizedlines)

punctuation = list(string.punctuation)
punctuation.extend(['--', '—', " – ", ' —', '«', '»'," -- ", " – – ", "\n", "\r", "\t","-" ,"-", "...", "…", " - ", " « ", " ", "..", "``", "\"\"","\'\'"])

def cleantext(text):
    tokens = text.split(" ")
    tokenslower = [i.strip().lower() for i in tokens]
#the following list comprehension gets rid of word-final punctuation, while retaining word-medial punctuation in words like "кто-то"
    tokens_no_word_final_punc = [re.sub(r'[!|,|?|.|:|;]|\.{1,}$\)', "", item) for item in tokenslower]
    tokens_no_stop_or_punc = [i for i in tokens_no_word_final_punc if i not in stop if i not in punctuation]
#the following line gets rid of blank, empty spaces in each list    
    tokens_no_stop_or_punc = list(filter(None, tokens_no_stop_or_punc))
    return tokens_no_stop_or_punc

newsocreal["no_punc_or_stop"] = newsocreal["text"].apply(cleantext)
cleaned = newsocreal["no_punc_or_stop"]

mystem = Mystem() 

def lemmatize(text):
    joinedtokens = " ".join(text)
    lemmatized = mystem.lemmatize(joinedtokens)
    lemmatized = [i for i in lemmatized if i not in stop if i not in punctuation]
    return lemmatized

newsocreal["lemmatized"] = newsocreal["no_punc_or_stop"].apply(lemmatize)

#Topic Modeling with Gensim, using Latent Semantic Indexing
dictionary = corpora.Dictionary(newsocreal["lemmatized"] )
doc_term_matrix = [dictionary.doc2bow(item) for item in newsocreal["lemmatized"] ]
lsi = models.LsiModel(doc_term_matrix, num_topics=4, id2word=dictionary)
output = lsi.print_topics(num_topics=4, num_words = 4)

print (output)




[(0, '-0.268*"человек" + -0.246*"рука" + -0.218*"глаз" + -0.158*"голова"'), (1, '-0.416*"мать" + -0.205*"человек" + 0.173*"крепость" + 0.165*"дело"'), (2, '-0.176*"крепость" + -0.167*"дело" + 0.158*"голова" + 0.155*"казак"'), (3, '-0.335*"товарищ" + -0.191*"завод" + -0.191*"рабочий" + 0.140*"крепость"')]


What happens with Latent Dirichlet Allocation (LDA) as opposed to LSI?

In [9]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

ldamodel = gensim.models.ldamodel.LdaModel(doc_term_matrix, num_topics=3, id2word = dictionary, passes=20)
ldaoutput = ldamodel.print_topics(num_topics=3, num_words=4)
print (ldaoutput)

[(0, '0.005*"товарищ" + 0.004*"рука" + 0.004*"человек" + 0.004*"глаз"'), (1, '0.005*"рука" + 0.005*"глаз" + 0.004*"голова" + 0.004*"человек"'), (2, '0.008*"человек" + 0.006*"рука" + 0.005*"глаз" + 0.004*"лицо"')]


In [10]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
2      46.739731        1       1  0.030533  0.066625
1      33.112774        1       2 -0.078488 -0.009180
0      20.147495        1       3  0.047955 -0.057445, topic_info=      Category        Freq           Term        Total  loglift  logprob
term                                                                    
1368   Default  874.000000        товарищ   874.000000  30.0000  30.0000
5609   Default  376.000000          казак   376.000000  29.0000  29.0000
6010   Default  288.000000          завод   288.000000  28.0000  28.0000
3012   Default  467.000000        рабочий   467.000000  27.0000  27.0000
1895   Default  838.000000           мать   838.000000  26.0000  26.0000
3942   Default  273.000000           гора   273.000000  25.0000  25.0000
1099   Default  374.000000         работа   374.000000  24.0000  24.0000
854    Default  570.000000          очень   570.000000  23.0000  23.0000
6057   Default  116.000000         колхоз   116.000000  22.0000  22.0000
13861  Default   68.000000        чумалов    68.000000  21.0000  21.0000
16671  Default  148.000000         дронов   148.000000  20.0000  20.0000
1310   Default  961.000000         стоять   961.000000  19.0000  19.0000
2444   Default  174.000000          хутор   174.000000  18.0000  18.0000
16643  Default  143.000000        варавка   143.000000  17.0000  17.0000
7987   Default  186.000000        станица   186.000000  16.0000  16.0000
11721  Default  211.000000       крепость   211.000000  15.0000  15.0000
521    Default  430.000000        красный   430.000000  14.0000  14.0000
9623   Default   86.000000        бригада    86.000000  13.0000  13.0000
973    Default  819.000000       понимать   819.000000  12.0000  12.0000
21392  Default   87.000000         андрий    87.000000  11.0000  11.0000
598    Default  468.000000          место   468.000000  10.0000  10.0000
16057  Default  154.000000           царь   154.000000   9.0000   9.0000
507    Default  403.000000           конь   403.000000   8.0000   8.0000
16055  Default  163.000000          хохол   163.000000   7.0000   7.0000
12420  Default  104.000000            бык   104.000000   6.0000   6.0000
7740   Default   55.000000  строительство    55.000000   5.0000   5.0000
175    Default  773.000000          время   773.000000   4.0000   4.0000
627    Default  746.000000           мочь   746.000000   3.0000   3.0000
7303   Default  194.000000           море   194.000000   2.0000   2.0000
609    Default  371.000000         минута   371.000000   1.0000   1.0000
...        ...         ...            ...          ...      ...      ...
1310    Topic3  230.099719         стоять   961.432030   0.1722  -5.8111
7303    Topic3   86.075873           море   194.773431   0.7855  -6.7944
598     Topic3  143.154312          место   468.363781   0.4168  -6.2857
175     Topic3  190.859264          время   773.188163   0.2031  -5.9981
1167    Topic3  325.774759           рука  1968.229433  -0.1966  -5.4634
269     Topic3  220.519205           дело  1088.299772   0.0057  -5.8536
1484    Topic3  315.830217        человек  2125.910841  -0.3047  -5.4944
426     Topic3  228.139550           идти  1223.595131  -0.0775  -5.8196
224     Topic3  274.874223           глаз  1741.281296  -0.2440  -5.6333
302     Topic3  139.836986           друг   511.964452   0.3043  -6.3091
2028    Topic3  127.660797           окно   437.443033   0.3705  -6.4002
567     Topic3  202.406389           лицо  1207.856206  -0.1842  -5.9393
627     Topic3  159.697120           мочь   746.404881   0.0601  -6.1763
232     Topic3  203.947796         голова  1269.240320  -0.2262  -5.9317
254     Topic3  154.735924         давать   725.770136   0.0566  -6.2079
609     Topic3  110.731291         минута   371.841976   0.3907  -6.5425
2452    Topic3  109.950809            час   376.797841   0.3704  -6.5496
1179    Topic3  140.158286          с

## Now, let's examine the banned literature corpus. 

In [6]:
bannedlit = pd.read_csv("csv_files/banned_lit_for_tm_smaller_chunks.csv")
bannedlit.head(5)

,title,name,date,english metadata,source,text
0,Собачье сердце,"Булгаков , Михаил",1925,"Mikhail Bulgakov , A Dog ’ s Heart , 1924",http : //bulgakov.lit-info.ru/bulgakov/proza/...,"У-у-у-у-у-гу-гуг-гуу ! О , гляньте на меня , ..."
1,\n\nСобачье сердце\n\n,"Булгаков, Михаил\n",2,"\n\nMikhail Bulgakov, A Dog’s Heart, 1924\n\n\n",\n\nhttp://bulgakov.lit-info.ru/bulgakov/proza...,", дорогой профессор , только в виде опыта . - ..."
2,\n\nСобачье сердце\n\n,"Булгаков, Михаил\n",3,"\n\nMikhail Bulgakov, A Dog’s Heart, 1924\n\n\n",\n\nhttp://bulgakov.lit-info.ru/bulgakov/proza...,Ястребиные ноздри его раздувались . Набравшись...
3,\n\nСобачье сердце\n\n,"Булгаков, Михаил\n",4,"\n\nMikhail Bulgakov, A Dog’s Heart, 1924\n\n\n",\n\nhttp://bulgakov.lit-info.ru/bulgakov/proza...,правом боку следы совершенно зажившего ожога ....
4,\n\nСобачье сердце\n\n,"Булгаков, Михаил\n",5,"\n\nMikhail Bulgakov, A Dog’s Heart, 1924\n\n\n",\n\nhttp://bulgakov.lit-info.ru/bulgakov/proza...,трубку в рогульки . Голубая радость разлилась ...


In [8]:
with open('NERoutputBanned.txt') as fp:
    lines = fp.read().splitlines()

lines = [i.lower() for i in lines]
joinedlines = " ".join(lines)
mystem = Mystem() 
lemmatizedlines = mystem.lemmatize(joinedlines)
lemmatizedlines = [i for i in lemmatizedlines if i != " "]

stop = list(stopwords.words('russian'))
stop.extend(['это', 'свой', 'то', ' ','который', 'говорить', 'сказать', 'знать', 'спрашивать', 'вопрос'])
stop.extend(lemmatizedlines)

punctuation = list(string.punctuation)
punctuation.extend(['--', '—', " – ", ' —', '«', '»'," -- ", " – – ", "\n", "\r", "\t","-" ,"-", "...", "…", " - ", " « ", " ", "..", "``", "\"\"","\'\'"])

def cleantext(text):
    tokens = text.split(" ")
    tokenslower = [i.strip().lower() for i in tokens]
#the following list comprehension gets rid of word-final punctuation, while retaining word-medial punctuation in words like "кто-то"
    tokens_no_word_final_punc = [re.sub(r'[!|,|?|.|:|;]|\.{1,}$\)', "", item) for item in tokenslower]
    tokens_no_stop_or_punc = [i for i in tokens_no_word_final_punc if i not in stop if i not in punctuation]
#the following line gets rid of blank, empty spaces in each list    
    tokens_no_stop_or_punc = list(filter(None, tokens_no_stop_or_punc))
    return tokens_no_stop_or_punc

bannedlit["no_punc_or_stop"] = bannedlit["text"].apply(cleantext)
cleaned = bannedlit["no_punc_or_stop"]

mystem = Mystem() 

def lemmatize(text):
    joinedtokens = " ".join(text)
    lemmatized = mystem.lemmatize(joinedtokens)
    lemmatized = [i for i in lemmatized if i not in stop if i not in punctuation]
    return lemmatized

bannedlit["lemmatized"] = bannedlit["no_punc_or_stop"].apply(lemmatize)

#Topic Modeling with Gensim, using Latent Semantic Indexing
dictionaryb = corpora.Dictionary(bannedlit["lemmatized"] )
doc_term_matrixb = [dictionaryb.doc2bow(item) for item in bannedlit["lemmatized"] ]
lsi = models.LsiModel(doc_term_matrixb, num_topics=4, id2word=dictionaryb)
output = lsi.print_topics(num_topics=4, num_words = 4)

print (output)


[(0, '0.332*"человек" + 0.184*"рука" + 0.176*"глаз" + 0.154*"жизнь"'), (1, '-0.275*"человек" + 0.220*"профессор" + 0.176*"глаз" + -0.159*"жизнь"'), (2, '0.362*"профессор" + 0.218*"человек" + 0.215*"отвечать" + -0.165*"наш"'), (3, '0.206*"год" + 0.182*"человек" + 0.146*"отец" + -0.137*"колхоз"')]


Now, what happens with LDA?

In [15]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

ldamodel = gensim.models.ldamodel.LdaModel(doc_term_matrixb, num_topics=4, id2word = dictionaryb, passes=20)
ldaoutput = ldamodel.print_topics(num_topics=3, num_words=4)
print (ldaoutput)

[(0, '0.005*"человек" + 0.004*"рука" + 0.004*"глаз" + 0.004*"профессор"'), (2, '0.009*"человек" + 0.004*"жизнь" + 0.004*"весь" + 0.004*"становиться"'), (1, '0.006*"глаз" + 0.006*"рука" + 0.004*"стена" + 0.004*"i"')]


In [16]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, doc_term_matrixb, dictionaryb)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
2      47.342695        1       1 -0.051203 -0.004676
0      26.398423        1       2 -0.016523 -0.087657
3      16.867803        1       3 -0.054007  0.073563
1       9.391079        1       4  0.121733  0.018771, topic_info=      Category        Freq         Term        Total  loglift  logprob
term                                                                  
204    Default  774.000000         глаз   774.000000  30.0000  30.0000
1011   Default  298.000000    профессор   298.000000  29.0000  29.0000
1088   Default  796.000000         рука   796.000000  28.0000  28.0000
1201   Default  196.000000        стена   196.000000  27.0000  27.0000
452    Default  404.000000     какой-то   404.000000  26.0000  26.0000
8462   Default   78.000000            i    78.000000  25.0000  25.0000
2345   Default   99.000000      древний    99.000000  24.0000  24.0000
928    Default  372.000000     понимать   372.000000  23.0000  23.0000
671    Default  359.000000          наш   359.000000  22.0000  22.0000
1436   Default   94.000000         ясно    94.000000  21.0000  21.0000
253    Default  291.000000        дверь   291.000000  20.0000  20.0000
1098   Default  407.000000        самый   407.000000  19.0000  19.0000
1082   Default   92.000000      розовый    92.000000  18.0000  18.0000
223    Default  123.000000         гора   123.000000  17.0000  17.0000
622    Default  499.000000         мочь   499.000000  16.0000  16.0000
13080  Default   85.000000        балка    85.000000  15.0000  15.0000
4530   Default  249.000000       солнце   249.000000  14.0000  14.0000
268    Default  573.000000         день   573.000000  13.0000  13.0000
1173   Default  171.000000   совершенно   171.000000  12.0000  12.0000
241    Default  120.000000         губа   120.000000  11.0000  11.0000
7761   Default   58.000000       единый    58.000000  10.0000  10.0000
4784   Default  144.000000         море   144.000000   9.0000   9.0000
1393   Default  193.000000  чувствовать   193.000000   8.0000   8.0000
8148   Default   62.000000  государство    62.000000   7.0000   7.0000
214    Default  479.000000       голова   479.000000   6.0000   6.0000
1865   Default  420.000000     отвечать   420.000000   5.0000   5.0000
456    Default  121.000000       камень   121.000000   4.0000   4.0000
431    Default  107.000000       именно   107.000000   3.0000   3.0000
2129   Default   75.000000   стеклянный    75.000000   2.0000   2.0000
843    Default   99.000000          пес    99.000000   1.0000   1.0000
...        ...         ...          ...          ...      ...      ...
6683    Topic4   32.441592        степа    67.578627   1.6316  -6.3890
2869    Topic4   28.668898        вверх    57.486611   1.6697  -6.5126
5070    Topic4   19.971246         бюро    33.648557   1.8437  -6.8741
126     Topic4   34.055379         вниз    76.669551   1.5539  -6.3404
241     Topic4   41.976661         губа   120.223444   1.3132  -6.1313
204     Topic4  113.977340         глаз   774.832693   0.4488  -5.1324
272     Topic4   25.950497        дикий    54.994988   1.6144  -6.6122
2827    Topic4   28.271621       улыбка    64.555913   1.5397  -6.5266
786     Topic4   25.030500       острый    51.951521   1.6352  -6.6483
1088    Topic4  106.331230         рука   796.088060   0.3523  -5.2018
452     Topic4   72.822219     какой-то   404.705501   0.6503  -5.5804
928     Topic4   68.753664     понимать   372.054009   0.6769  -5.6379
671     Topic4   63.055732          наш   359.957531   0.6234  -5.7244
1098    Topic4   64.566016        самый   407.840896   0.5222  -5.7007
2083    Topic4   36.374891       сквозь   117.125196   1.1960  -6.2745
622     Topic4   68.642885         мочь   499.201778   0.3813  -5.6395
431     Topic4   34.597107       именно   107.265311   1.2339  -6.3246
268     Topic4   58.064622         день   573.406981   0.0754  -5.8069
102     Topi